## Term Project Milestone 4
### DSC 540
#### Milan Sherman

A developer actually a created college football data (cfbd) package, which was very convenient.  I just had to spend some time looking over the docs in order to understand the classes and methods available and what I needed to use to get the data that I needed.  The data I'm pulling from the API are the final college football rankings for the 2010-2021 seasons, as those are the seasons that existed in my flat file data source.  Quick summary of my data sources:
- Offensive statistics for every college football team from 2010-2021 (flat file)
- Defensive statistics for every college football team from 2010-2021 (web scraping)
- Top 25 Final Rankings for 2010-2021 (API)
So the final dataframe from this data source will be the Top 25 Teams in the final rankings for each season 2010-2021.  I will left join this data on to a combination of the above two data sources, as each of those sources rank every team while this data will only have the Top 25.

In [1]:
import cfbd
# create a configuration object to configure the api authorization using the api key
config = cfbd.Configuration()
config.api_key['Authorization'] = '3BTTYOSaARYTxZLkq1SeyIazENP+SHb9e4/VwdAH8i+5vYZ1SZ/phIK2PVS+J+le'
config.api_key_prefix['Authorization'] = 'Bearer'

# instantiate the object
api_instance = cfbd.RankingsApi(cfbd.ApiClient(config))
# create lists for the season and rankings
poll=[]
season=[]
final_poll=[]
# I am pulling the rankings for years 2010-2021
# postseason, week 1 gets the final rankings for the seaon
for i in range(0, 12):
    rank=api_instance.get_rankings(year=i+2010, season_type = 'postseason', week=1)
# rank is a list object, and rank[0] is a RankingWeek object with methods for season and poll
# season gives the year of the season, and polls yields a dictionary of dictionaries,
# where each dictionary is a different ranking/polling system, e.g. AP Top 25, Coaches Poll, Playoff Committee Rankings, etc.
    
#     I save each season to add this as a column in the final df
    season.append(rank[0].season)
    ranking=rank[0].polls
    for j in range(len(ranking)):
#         for each season, get the poll/ranking system, and the rankings by team for that poll
        final_poll.append(ranking[j])

In [2]:
# I really wanted the final rankings based on the Playoff Committee Ranking system
# but the playoffs didn't start until 2014
# the AP Top 25 was the only poll/ranking system that we included every year, and I think it has aligned with 
# the Playoff Committee Ranking poll since the playoffs have started, so that is the poll I'm using
ap25=[]
for i in range(len(final_poll)):
    if final_poll[i]['poll']=='AP Top 25':
        ap25.append(final_poll[i]['ranks'])

In [3]:
# add the season to the dictionary for each seaon/team
for i in range(len(ap25)):
    for j in range(len(ap25[i])):
        ap25[i][j].update({'season':season[i]})

In [4]:
import pandas as pd
# create a dataframe for each year from the team/season dictionary
# create a list of dataframes, one for each year
dfs=[]
for i in range(len(ap25)):
    df=pd.DataFrame.from_records(ap25[i])
    df.drop(['firstPlaceVotes', 'points'], axis=1, inplace=True)
    dfs.append(df)

In [5]:
# concatenate the 12 dataframes into a single dataframe
rankings = pd.concat(dfs)
rankings

,rank,school,conference,season
0,1,Auburn,SEC,2010
1,2,TCU,Mountain West,2010
2,3,Oregon,Pac-10,2010
3,4,Stanford,Pac-10,2010
4,5,Ohio State,Big Ten,2010
...,...,...,...,...
20,21,Arkansas,SEC,2021
21,22,Oregon,Pac-12,2021
22,23,Iowa,Big Ten,2021
23,24,Utah State,Mountain West,2021


In [6]:
# Check data types
rankings.dtypes

rank           int64
school        object
conference    object
season         int64
dtype: object

These look right - integer for the rank (1-25) and season (2010-2021), and character for school name and conference

Below I rename the columns to align with the dataframes from my flat file and web scraping sources to make it easier to join

In [7]:
rankings = rankings.rename(columns = {'rank':'Final Rank','school':'Team','conference':'Conference','season':'Year'})
rankings.head()

,Final Rank,Team,Conference,Year
0,1,Auburn,SEC,2010
1,2,TCU,Mountain West,2010
2,3,Oregon,Pac-10,2010
3,4,Stanford,Pac-10,2010
4,5,Ohio State,Big Ten,2010


I'm not really sure what else I can do with this in terms of transformations.  The data types are right, and the headers are set up to make merging with my other data sources easier.  Most of my time was spent getting familiar with the classes and methods for cfbd package, understanding the structure of the data (list of dictionary of dictionaries), and unpacking/unraveling the data that I wanted (Final Top 25 for each year).  

My goal is to create a dataset that will allow analysis of a variety of offensive/defensive statistics to determine which are most predictive of a team's final ranking.

In [8]:
import os
os.getcwd()

'C:\\Users\\milan\\Documents\\GitHub\\DS-540\\Term Project'

In [9]:
rankings.to_csv('Final Rankings.csv')